In [76]:
DL = '/Users/damiansp/Downloads'

In [77]:
!rm -rf $DL/*

In [78]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [79]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ACN', 'ADBE', 'ADYEY', 'AMD', 'AMGN',
    'AMZN', 'ANET', 'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD',
    'DDOG', 'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH',
    'GMED', 'GOOG', 'IDXX', 'INTG', 'ISRG', 'JD', 'LRCX', 'LSPD',
    'LULU', 'MDB', 'MELI', 'MNST', 'MSFT', 'MTCH', 'NVCR', 'NVDA',
    'ODFL', 'OKTA', 'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS', 'PTON',
    'PYPL', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'RIVN', 'RMD', 'ROKU',
    'RVLV', 'SE', 'SHOP', 'SNOW', 'TASK', 'TDOC', 'TEAM', 'TREX',
    'TSCO', 'TSLA', 'TTD', 'TTWO', 'TWLO', 'TWTR', 'TYL', 'UPST',
    'VEEV', 'VLD', 'VRTX', 'XPEV', 'ZEN', 'ZM']
LINGERERS = [
    'AMEH', 'AMRK', 'APT', 'CENN', 'CTGO', 'CUBE',
    'DTST', 'EP', 'FCPT', 'FCUV', 'FKWL', 'GGE', 'GME', 
    'INTZ', 'KREF', 'LFMD', 'MCVT', 'MIY', 'MNMD', 'MOBQ',
    'PETV', 'POWW', 'PRPH', 'RVYL', 'VIRT', 'WAVD', 'WKHS',
    'ZYXI']
BEST_WEIGHTED = [
    'BTTR', 'BYRN', 'CEF', 'CGAU', 'CIZN', 'DYAI', 'HMY', 'HNRG',
    'IEI', 'MGF', 'MICS', 'MMU', 'MUE', 'NAN', 'NOM', 'NXN', 'NXP',
    'OMQS', 'RVP', 'SOBR']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))

In [80]:
!ls $DL

In [81]:
#!head -5 $DL/Positions.csv

In [82]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [83]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [84]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ACN
ADBE


### E*Trade

In [85]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [86]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
90184L102,--
AAPL,1557.4
ABMD,1548.3600000000001
ABNB,230.42
ADBE,1628.4


In [87]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
90184L102,--
AAPL,1557
ABMD,1548
ABNB,230
ADBE,1628


In [88]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [96]:
et_df.loc[et_df.ET == '--']
#et_df.loc['90184L102', 'ET'] = 0

,ET


In [95]:
check_symbols(et_df)

All clear


In [94]:
#et_df = et_df[1:]

In [97]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,1557
ABMD,1548
ABNB,230
ACN,0
ADBE,1628


### Fidelity

In [98]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Oct-28-2022.csv'

In [99]:
!ls $DL

Portfolio_Positions_Oct-28-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_10_28.xls


In [100]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-24-", "-25-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,Rollover IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,11.97,$1.00,$0.00,$11.97,NaN,NaN,NaN,NaN,0.04%,NaN,NaN,Cash
1,218520300,Rollover IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$258.06,-$9.52,$1032.24,-$38.08,-3.56%,+$13.55,+1.33%,3.02%,$1018.69,$254.67,Cash
2,218520300,Rollover IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.00,$14.47,-$0.064,$188.11,-$0.84,-0.45%,-$26.58,-12.39%,0.55%,$214.69,$16.51,Cash
3,218520300,Rollover IRA,AMD,ADVANCED MICRO DEVICES INC,18.00,$62.01,+$3.41,$1116.18,+$61.38,+5.81%,-$230.73,-17.14%,3.27%,$1346.91,$74.83,Cash
4,218520300,Rollover IRA,AMZN,AMAZON.COM INC,2.00,$103.41,-$7.55,$206.82,-$15.10,-6.81%,-$69.95,-25.28%,0.61%,$276.77,$138.39,Cash


In [101]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,Rollover IRA,ABMD,1032
2,Rollover IRA,ADYEY,188
3,Rollover IRA,AMD,1116
4,Rollover IRA,AMZN,207
5,Rollover IRA,AXON,426


In [102]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['Rollover IRA'].head()

,Rollover IRA
Symbol,
ABMD,1032
ADYEY,188
AMD,1116
AMZN,207
AXON,426


In [103]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

Rollover IRA
All clear

ROTH IRA
90184L102 not in SYMBOLS

SIMPLE IRA
All clear



In [104]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA
AAPL,1557,0,6230,0
ABMD,1548,1032,3355,0
ABNB,230,0,576,0
ACN,0,0,0,0
ADBE,1628,0,0,0


### TD Ameritrade

In [105]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [106]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

In [107]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1])
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [108]:
td = parse_file(path)
td.head()

,TD
AMEH,70.32
AMRK,120.48
APT,91.08
BROS,36.42
BTTR,114.33


In [109]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,1557,0,6230,0,0
ABMD,1548,1032,3355,0,0
ABNB,230,0,576,0,0
ACN,0,0,0,0,0
ADBE,1628,0,0,0,0


In [110]:
out.to_csv(f'{DL}/clean.csv')

In [111]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*